### Алгоритм первичный для тестов
На каждом шаге
1. считаем величину 
$$w = (X^T X)^{-1}X^T y$$
2. Строим модель (линейную или квадратичную)
$$ model = w_0 + w_1 price $$
3. СКО модел
$$ error = price - model $$
$$ \sigma = \sqrt{\frac{error^2}{N-2}} $$
4. Совершаем сделки
- if $price > k\sigma$: then short  
    do while:
        price <= model
- if $price < k\sigma$: then long  
    do while:
        price => model

In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
# backtesting.set_bokeh_output(notebook=False)
import pandas as pd
import talib
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from src.ramsy import ramsey_reset_test
from statsmodels.stats.stattools import durbin_watson
from src.het import het_breuschpagan
from scipy.stats import shapiro
import os


In [20]:
p = "/home/strike/work/external/trade/mm_profect/general_reseach/regress/data_csv/"
d = os.listdir(p)
d.sort()
d

['20240101-20240131',
 '20240201-20240229',
 '20240301-20240331',
 '20240401-20240430',
 '20240501-20240531',
 '20240601-20240630',
 '20240701-20240731',
 '20240801-20240831',
 '20240901-20240930',
 '20241001-20241028']

In [21]:
# df = pd.read_csv(    "/home/strike/work/external/trade/mm_profect/general_reseach/regress/15_sec.csv")
df = pd.read_csv(p + d[5])
print(df)
df = df.dropna()  # .reset_index(drop=True)
# from datetime import datetime
df["ts_event"] = pd.to_datetime(df["ts_event"])
df["ts_event"] = df["ts_event"].dt.tz_localize(None)
df.set_index("ts_event", inplace=True)
# df = df[:10000]
data = df.loc[:, ["open", "high", "low", "close"]].rename(
    columns={"open": "Open", "high": "High", "low": "Low", "close": "Close"}
)

                         ts_event      open      high       low     close  \
0       2024-06-02 22:00:00+00:00  18590.25  18595.75  18576.00  18587.25   
1       2024-06-02 22:00:15+00:00  18588.00  18595.25  18587.75  18593.25   
2       2024-06-02 22:00:30+00:00  18592.75  18595.50  18586.50  18588.25   
3       2024-06-02 22:00:45+00:00  18588.00  18598.50  18587.75  18595.25   
4       2024-06-02 22:01:00+00:00  18594.25  18597.00  18591.50  18595.50   
...                           ...       ...       ...       ...       ...   
107395  2024-06-21 13:28:45+00:00  19760.50  19760.50  19758.50  19758.50   
107396  2024-06-21 13:29:00+00:00  19762.75  19763.75  19762.75  19763.75   
107397  2024-06-21 13:29:15+00:00  19770.75  19770.75  19763.75  19770.75   
107398  2024-06-21 13:29:30+00:00  19767.75  19771.25  19758.50  19758.50   
107399  2024-06-21 13:29:45+00:00  19756.00  19756.00  19756.00  19756.00   

        size_size  
0             423  
1             177  
2             1

In [24]:
df1 = data[: int(len(df) * 3 / 4)]
df2 = data[int(len(df) * 3 / 4) :]


In [29]:
data = df2

In [32]:
data

,Open,High,Low,Close
ts_event,,,,
2024-06-16 22:50:30,19717.25,19717.25,19717.25,19717.25
2024-06-16 22:50:45,19717.25,19717.75,19717.25,19717.75
2024-06-16 22:51:00,19718.25,19718.25,19718.25,19718.25
2024-06-16 22:51:15,19718.50,19718.75,19718.50,19718.50
2024-06-16 22:51:30,19718.75,19719.00,19718.75,19719.00
...,...,...,...,...
2024-06-21 13:28:45,19760.50,19760.50,19758.50,19758.50
2024-06-21 13:29:00,19762.75,19763.75,19762.75,19763.75
2024-06-21 13:29:15,19770.75,19770.75,19763.75,19770.75


In [ ]:
"""
x = np.linspace(0, 1, num=window)
one = np.ones_like(x)
X = np.vstack([one, x]).T
t_lin = np.linalg.inv(X.T @ X) @ X.T
"""


class RegressionStrategy(Strategy):
    window = 202  # Количество свечей
    k = 35
    p_value_thresh = 0.1  # Порог стационарности

    def init(self):
        pass

    def next(self):
        # Пропускаем первые window свечей
        if len(self.data.Close) < self.window:
            return

        x = np.linspace(0, 1, num=self.window)
        one = np.ones_like(x)
        X = np.vstack([one, x, x**2]).T
        self.t_lin = np.linalg.inv(X.T @ X) @ X.T
        self.x = x

        # Берем последние window цен закрытия
        closes = self.data.Close[-self.window :]
        # Создаем массив индексов для регрессии
        # x = np.arange(len(closes))
        """
        result = adfuller(closes)
        p_value = result[1]
        
        # Если ряд не стационарен - не торгуем
    
        if p_value >= self.p_value_thresh:
            return
            """

        if not self.position:
            # Тест Рамсея на линейность
            _, s = ramsey_reset_test(self.x, closes - np.mean(closes))
            X = self.x.reshape(-1, 1)
            _, p_value = het_breuschpagan(X, closes)
            if (p_value < 0.1) * (s < 0.1):  # * (s < 0.1)
                return

        # тест на гетероскедантичность
        """
        X = self.x.reshape(-1,1)
        _, p_value = het_breuschpagan(X, closes)
        if (p_value>0.1) * (not self.position):
            return
        """

        # Рассчитываем коэффициенты линейной регрессии
        w = self.t_lin @ closes

        slope = w[1]
        intercept = w[0]
        # slope, intercept = np.polyfit(x, closes, 1)

        # Получаем линию регрессии
        regression_line = slope * self.x + intercept  # + w[2]*self.x**2

        # Рассчитываем СКО ошибок
        residuals = closes - regression_line

        # Тест на нормальность

        if not self.position:
            _, p_value = shapiro(residuals)
            if p_value < 0.1:
                return

        # Дарли Уотсона тест на постояноство ошибки (гетероскедантичность)
        """
        if (not self.position) * (durbin_watson(residuals)<0.7):
            return
            """
        std = np.std(residuals / (self.window - 2))

        # Текущие значения
        current_reg = regression_line[-1]
        current_price = closes[-1]
        upper_band = current_reg + std * self.k
        lower_band = current_reg - std * self.k

        # Логика торговли
        if not self.position:
            # Сигнал на продажу
            if current_price > upper_band:
                self.sell()

            # Сигнал на покупку
            elif current_price < lower_band:
                self.buy()

        else:
            # Выход из длинной позиции
            if self.position.is_long and current_price > current_reg:
                self.position.close()

            # Выход из короткой позиции
            elif self.position.is_short and current_price < current_reg:
                self.position.close()


# Запуск бэктеста
bt = Backtest(data, RegressionStrategy, cash=100000, commission=0.0000)

stats = bt.optimize(
    k=range(20, 100),  # Перебираем целые значения от 1 до 100
    window=range(30, 300),
    maximize="Sharpe Ratio",  # maximize="Equity Final [$],  # Критерий оптимизации - конечный баланс
    method="grid",  # Метод перебора
    max_tries=100,  # Лимит количества комбинаций (можно увеличить)
)


# stats = bt.run()
# print(stats._strategy)
print(stats)
bt.plot()

Backtest.optimize:   0%|          | 0/100 [00:00<?, ?it/s]

Start                     2024-06-16 22:50:30
End                       2024-06-21 13:29:45
Duration                      4 days 14:39:15
Exposure Time [%]                    74.60881
Equity Final [$]                     100707.5
Equity Peak [$]                      101092.5
Return [%]                             0.7075
Buy & Hold Return [%]                 0.19653
Return (Ann.) [%]                    33.61659
Volatility (Ann.) [%]                 2.73827
CAGR [%]                              47.0107
Sharpe Ratio                          12.2766
Sortino Ratio                             inf
Calmar Ratio                         36.10502
Max. Drawdown [%]                    -0.93108
Avg. Drawdown [%]                    -0.04344
Max. Drawdown Duration        2 days 12:53:00
Avg. Drawdown Duration        0 days 01:05:26
# Trades                                  607
Win Rate [%]                         60.95552
Best Trade [%]                        0.19922
Worst Trade [%]                   

/home/strike/work/penv/deep/lib/python3.13/site-packages/backtesting/_plotting.py:139: UserWarning: Data contains too many candlesticks to plot; downsampling to '1min'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


GridPlot(id='p1719', ...)